# English Learning App - Word Sense Filtering
This notebook filters words from song lyrics that don't match their intended semantic meaning based on translation into Hebrew.

In [ ]:
import pandas as pd
import ast
import re
import os
from sentence_transformers import SentenceTransformer, util
from googletrans import Translator
import time
from difflib import SequenceMatcher
from category_in_hebrew import intended_hebrew

# --- General Configuration ---
BATCH_SIZE = 1000
INPUT_PATH = '../data/songs_category_before_tran.csv'
OUTPUT_PATH = '../data/filtered_songs_disambiguated.csv'

# --- Load model and translation tools ---
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
translator = Translator()
translation_cache = {}

# Retry mechanism for Google Translate to handle rate limits/errors
def translate_with_retry(sentence, retries=3, delay=1):
    for attempt in range(retries):
        try:
            return translator.translate(sentence, src='en', dest='he').text
        except Exception as e:
            print(f"Translation failed (attempt {attempt+1}): {e}")
            time.sleep(delay)
    return None

# Use cache to avoid redundant translations
def translate_with_cache(sentence):
    if sentence in translation_cache:
        return translation_cache[sentence]
    translated = translate_with_retry(sentence)
    if translated:
        translation_cache[sentence] = translated
    return translated

# Basic sentence splitter
def split_sentences(text):
    return re.split(r'[.!?\n]', text)

# Check if the word is used in the intended sense
def is_correct_sense(word, sentence, intended_hebrew_meaning, threshold=0.75):
    translated = translate_with_cache(sentence)
    if not translated:
        return False

    # Direct match
    if intended_hebrew_meaning in translated:
        print(f"\n✔️ {word.upper()} found via direct match: '{intended_hebrew_meaning}' in '{translated}'")
        return True

    # Fuzzy string match
    similarity_str = SequenceMatcher(None, translated, intended_hebrew_meaning).ratio()
    if similarity_str > 0.85:
        print(f"\n✔️ {word.upper()} found via fuzzy string match ({similarity_str:.2f})")
        return True

    # Semantic similarity using sentence embeddings
    try:
        emb_sentence = model.encode(translated, convert_to_tensor=True)
        emb_meaning = model.encode(intended_hebrew_meaning, convert_to_tensor=True)
        similarity = util.pytorch_cos_sim(emb_sentence, emb_meaning).item()
        print(f"\nWord: {word}")
        print(f"Original Sentence: {sentence}")
        print(f"Translated (HE): {translated}")
        print(f"Intended (HE): {intended_hebrew_meaning}")
        print(f"Similarity: {similarity:.4f} — {'✔️ KEEP' if similarity >= threshold else '❌ SKIP'}")
        return similarity >= threshold
    except Exception as e:
        print(f"Embedding failed: {e}")
        return False

# --- Load data ---
df = pd.read_csv(INPUT_PATH)

# Load previously processed data if exists
if os.path.exists(OUTPUT_PATH):
    existing_df = pd.read_csv(OUTPUT_PATH)
    processed_count = len(existing_df)
else:
    existing_df = pd.DataFrame()
    processed_count = 0

print(f"Processed so far: {processed_count}")

# Select the next batch of songs to process
df_batch = df.iloc[processed_count:processed_count + BATCH_SIZE]

if df_batch.empty:
    print("✅ All songs processed.")
    exit()

filtered_rows = []

# --- Process current batch ---
for idx, row in df_batch.iterrows():
    lyrics = row['cleaned_lyrics'].lower()
    updated_groups = []
    category_groups = ast.literal_eval(row['category_words'])
    sentences = split_sentences(lyrics)

    for group in category_groups:
        filtered_group = []
        for word in group:
            found = False
            for category_dict in intended_hebrew.values():
                if word in category_dict:
                    intended_hebrew_meaning = category_dict[word]
                    sentence = next((s.strip() for s in sentences if word in s), None)
                    if sentence and is_correct_sense(word, sentence, intended_hebrew_meaning):
                        filtered_group.append(word)
                    found = True
                    break
            if not found:
                filtered_group.append(word)

        if filtered_group:
            updated_groups.append(filtered_group)

    row['category_words'] = str(updated_groups)
    filtered_rows.append(row)

# --- Save results ---
filtered_df = pd.DataFrame(filtered_rows)
combined_df = pd.concat([existing_df, filtered_df], ignore_index=True)
combined_df.to_csv(OUTPUT_PATH, index=False)

print(f"✅ Batch processed and saved: {processed_count} ➡ {processed_count + len(filtered_df)}")


/home/hila/Jingo-1/jupyter-env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



✔️ HATE found via direct match: 'שנאה' in 'זה מתחיל בכאב ואחריו שנאה'

✔️ SHAME found via direct match: 'בושה' in 'האם אתה שפוי איפה הבושה'

✔️ FEELING found via direct match: 'תחושה' in 'זו תחושה שאתה מקבל כשאתה לא יכול למצוא את הדרך שלך'

Word: kind
Original Sentence: what kind of world do we live in
Translated (HE): באיזה סוג עולם אנו חיים
Intended (HE): אדיב
Similarity: 0.7474 — ❌ SKIP

✔️ SANE found via direct match: 'שפוי' in 'האם אתה שפוי איפה הבושה'

✔️ LOVE found via direct match: 'אהבה' in 'שם האהבה מחולקת בשנאה'

✔️ SHAME found via direct match: 'בושה' in 'נמאס מכל הבושה שהרגשתי'

✔️ DOUBT found via direct match: 'ספק' in 'לעולם לא יהיה ספק'

✔️ TIRED OF found via direct match: 'נמאס' in 'נמאס מכל הבושה שהרגשתי'

Word: loved
Original Sentence: but you loved me anyway
Translated (HE): אבל בכל מקרה אהבת אותי
Intended (HE): אהוב
Similarity: 0.8470 — ✔️ KEEP

✔️ SICK found via direct match: 'חולה' in 'הייתי חולה מכל הכאב'

✔️ LOVE found via direct match: 'אהבה' in 'אהבה שונאת ג

In [4]:
input_file = '../data/filtered_songs_three.csv'
df = pd.read_csv(input_file)
print(f"Total songs in dataset: {input_file}: {len(df)}")

Total songs in dataset: ../data/filtered_songs_three.csv: 96515
